# Análisis Comparativo de Propuestas de Bonificación

# 1. Configuración Inicial

#### Importación de Bibliotecas y Configuración

In [1]:
import pandas as pd  
import os  

# Definir la ruta del directorio  
ruta_directorio = r'C:\Users\Francisco Villanueva\Desktop\TALLER_2'

# 2. Estructura Principal y Manejo de Errores

In [20]:
# Estructura principal del programa  
try:  
    # El código principal irá aquí en los siguientes pasos  
    pass  # Este pass es temporal y se reemplazará con el código real  
except Exception as e:  
    print(f"Error durante el análisis: {str(e)}")  

# 3. Carga y Limpieza de Datos

In [21]:
# Leer el archivo Excel original  
df = pd.read_excel(os.path.join(ruta_directorio, 'Base_control_2.xlsx'))  

# Limpieza básica de datos  
df_limpio = df.dropna(subset=['Nombre Ejecutivo'])  
df_limpio = df_limpio[  
    (df_limpio['Nombre Ejecutivo'] != 'n/d') &  
    (df_limpio['Nombre Ejecutivo'] != 'ejecutivo prueba') &  
    (~df_limpio['Nombre Ejecutivo'].str.contains('ejecutivo prueba', case=False, na=False))  
]  

# Convertir fecha y filtrar años  
df_limpio.iloc[:, 1] = pd.to_datetime(df_limpio.iloc[:, 1])  
df_limpio = df_limpio[df_limpio.iloc[:, 1].dt.year.isin([2023, 2024])]  

# 4. Definición de Funciones para Cálculo de Bonos

In [22]:
def calcular_bono_propuesta1(monto):  
    if monto < 50000000:  
        return monto * 0.0025  
    elif 50000000 <= monto <= 200000000:  
        return monto * 0.0050  
    else:  
        return monto * 0.0060  

def calcular_bono_propuesta2(ingreso):  
    if ingreso < 100000:  
        return ingreso * 0.0500  
    elif 100000 <= ingreso <= 1000000:  
        return ingreso * 0.0875  
    else:  
        return ingreso * 0.1000  

def calcular_bono_propuesta3(monto):  
    if monto < 50000000:  
        return monto * 0.0015  # 0.15%  
    elif 50000000 <= monto <= 200000000:  
        return monto * 0.0040  # 0.40%  
    else:  
        return monto * 0.0050  # 0.50%  

def calcular_bono_fijo_propuesta3(num_clientes):  
    if num_clientes < 8:  
        return 50000  
    elif 8 <= num_clientes <= 14:  
        return 120000  
    else:  
        return 350000

# 5. Aplicación de Cálculos y Preparación de Datos

In [23]:
# Aplicar cálculos de bonos y redondear a 2 decimales  
df_limpio['Bono_Propuesta_1'] = df_limpio['Monto factura'].apply(calcular_bono_propuesta1).round(2)  
df_limpio['Bono_Propuesta_2'] = df_limpio['Ingreso por operación'].apply(calcular_bono_propuesta2).round(2)  
df_limpio['Bono_Propuesta_3'] = df_limpio['Monto factura'].apply(calcular_bono_propuesta3).round(2)  

# Agregar año y mes  
df_limpio['Año'] = df_limpio.iloc[:, 1].dt.year  
df_limpio['Mes'] = df_limpio.iloc[:, 1].dt.month

# 6. Generación de Resumen Mensual

In [24]:
# Crear resumen mensual  
resumen_mensual = df_limpio.groupby(['Nombre Ejecutivo', 'Año', 'Mes']).agg({  
    'Monto factura': ['sum', 'count', 'mean'],  
    'Ingreso por operación': ['sum', 'mean'],  
    'Bono_Propuesta_1': 'sum',  
    'Bono_Propuesta_2': 'sum',  
    'Bono_Propuesta_3': 'sum',  
    'Nombre empresa solicitante': 'nunique'  # Este es el que cuenta clientes únicos  
}).reset_index()  

# Aplanar columnas multinivel  
resumen_mensual.columns = ['Ejecutivo', 'Año', 'Mes', 'Monto_Total',  
                        'Num_Operaciones', 'Monto_Promedio',  
                        'Ingreso_Total', 'Ingreso_Promedio',  
                        'Bono_1', 'Bono_2', 'Bono_3',  
                        'Num_Clientes_Unicos']  

# Convertir Num_Clientes_Unicos a entero  
resumen_mensual['Num_Clientes_Unicos'] = resumen_mensual['Num_Clientes_Unicos'].astype(int)

# 7. Cálculo de Bonos Totales

In [25]:
# Agregar bono fijo propuesta 3  
resumen_mensual['Bono_Fijo_Prop3'] = resumen_mensual['Num_Clientes_Unicos'].apply(  
    calcular_bono_fijo_propuesta3  
).round(2)  

# Calcular bonos totales  
resumen_mensual['Bono_Total_Prop3'] = (resumen_mensual['Bono_3'] + resumen_mensual['Bono_Fijo_Prop3']).round(2)

# 8. Análisis Estadístico

In [33]:
# Análisis comparativo de las tres propuestas  
comparativo = resumen_mensual.agg({  
    'Bono_1': ['mean', 'min', 'max', 'std'],  
    'Bono_2': ['mean', 'min', 'max', 'std'],  
    'Bono_Total_Prop3': ['mean', 'min', 'max', 'std']  
}).round(2)  

# Análisis por ejecutivo y año  
analisis_ejecutivo = resumen_mensual.groupby(['Ejecutivo', 'Año']).agg({  
    'Bono_1': 'sum',  
    'Bono_2': 'sum',  
    'Bono_Total_Prop3': 'sum',  
    'Num_Clientes_Unicos': 'max'  
}).astype({  
    'Num_Clientes_Unicos': 'int64'  
})  

# Redondeamos las columnas de bonos  
columnas_bonos = ['Bono_1', 'Bono_2', 'Bono_Total_Prop3']  
analisis_ejecutivo[columnas_bonos] = analisis_ejecutivo[columnas_bonos].round(2)  

# Calcular el total de las tres propuestas y crear ranking  
analisis_ejecutivo['Total_Bonos'] = analisis_ejecutivo[columnas_bonos].sum(axis=1).round(2)  

# Resetear el índice y ordenar por Total_Bonos de mayor a menor  
analisis_ejecutivo = analisis_ejecutivo.reset_index()  
analisis_ejecutivo = analisis_ejecutivo.sort_values(['Año', 'Total_Bonos'], ascending=[True, False])  

# Agregar columna de ranking por año  
analisis_ejecutivo['Ranking'] = analisis_ejecutivo.groupby('Año')['Total_Bonos'].rank(ascending=False, method='min').astype(int)  

# Reordenar las columnas para mejor visualización  
analisis_ejecutivo = analisis_ejecutivo[[  
    'Año',  
    'Ranking',  
    'Ejecutivo',  
    'Bono_1',  
    'Bono_2',  
    'Bono_Total_Prop3',  
    'Total_Bonos',  
    'Num_Clientes_Unicos'  
]]

# 9. Presentación de Resultados y Exportación 

In [34]:
# Definir ruta de archivo  
ruta_archivo = os.path.join(ruta_directorio, 'Analisis_Comparativo_Propuestas.xlsx')  

# Preparar el resumen para Excel  
resumen_comparativo = pd.DataFrame({  
    'Métrica': ['Promedio Mensual', 'Desviación Estándar'],  
    'Propuesta 1': [  
        f"${comparativo.loc['mean', 'Bono_1']:,.2f}",  
        f"${comparativo.loc['std', 'Bono_1']:,.2f}"  
    ],  
    'Propuesta 2': [  
        f"${comparativo.loc['mean', 'Bono_2']:,.2f}",  
        f"${comparativo.loc['std', 'Bono_2']:,.2f}"  
    ],  
    'Propuesta 3': [  
        f"${comparativo.loc['mean', 'Bono_Total_Prop3']:,.2f}",  
        f"${comparativo.loc['std', 'Bono_Total_Prop3']:,.2f}"  
    ]  
})  

# Identificar la mejor propuesta  
promedios = {  
    'Propuesta 1': comparativo.loc['mean', 'Bono_1'],  
    'Propuesta 2': comparativo.loc['mean', 'Bono_2'],  
    'Propuesta 3': comparativo.loc['mean', 'Bono_Total_Prop3']  
}  
mejor_propuesta = max(promedios, key=promedios.get)  

# Agregar la conclusión al resumen  
resumen_comparativo.loc[2] = ['Mejor Propuesta',   
                            mejor_propuesta if mejor_propuesta == 'Propuesta 1' else '',  
                            mejor_propuesta if mejor_propuesta == 'Propuesta 2' else '',  
                            mejor_propuesta if mejor_propuesta == 'Propuesta 3' else '']  

# Guardar todos los resultados en Excel  
with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:  
    # Resumen mensual detallado  
    resumen_mensual.to_excel(  
        writer,  
        sheet_name='Resumen_Mensual',  
        index=False  
    )  
    
    # Análisis por ejecutivo (ranking)  
    analisis_ejecutivo.to_excel(  
        writer,  
        sheet_name='Ranking_Ejecutivos',  
        index=False  
    )  
    
    # Comparativo general  
    comparativo.to_excel(  
        writer,  
        sheet_name='Comparativo_General'  
    )  
    
    # Resumen comparativo con conclusión  
    resumen_comparativo.to_excel(  
        writer,  
        sheet_name='Resumen_Final',  
        index=False  
    )  
    
    # Datos procesados  
    df_limpio.to_excel(  
        writer,  
        sheet_name='Datos_Procesados',  
        index=False  
    )  

# Mostrar mensaje de completado y resumen en consola  
print(f"\nAnálisis completado. Archivo guardado en: {ruta_archivo}")  
print("\nResumen Comparativo de Propuestas:")  
print("\nPromedios mensuales:")  
print(f"Bono Propuesta 1: ${comparativo.loc['mean', 'Bono_1']:,.2f}")  
print(f"Bono Propuesta 2: ${comparativo.loc['mean', 'Bono_2']:,.2f}")  
print(f"Bono Propuesta 3: ${comparativo.loc['mean', 'Bono_Total_Prop3']:,.2f}")  

print("\nDesviación estándar:")  
print(f"Propuesta 1: ${comparativo.loc['std', 'Bono_1']:,.2f}")  
print(f"Propuesta 2: ${comparativo.loc['std', 'Bono_2']:,.2f}")  
print(f"Propuesta 3: ${comparativo.loc['std', 'Bono_Total_Prop3']:,.2f}")  

print(f"\nLa {mejor_propuesta} ofrece el mejor bono promedio mensual")


Análisis completado. Archivo guardado en: C:\Users\Francisco Villanueva\Desktop\TALLER_2\Analisis_Comparativo_Propuestas.xlsx

Resumen Comparativo de Propuestas:

Promedios mensuales:
Bono Propuesta 1: $2,752,706.62
Bono Propuesta 2: $469,872.22
Bono Propuesta 3: $2,266,984.52

Desviación estándar:
Propuesta 1: $14,186,461.06
Propuesta 2: $464,200.30
Propuesta 3: $11,805,053.16

La Propuesta 1 ofrece el mejor bono promedio mensual
